In [ ]:
import pandas as pd
import os
import requests
from io import StringIO

estados_ibge = {
    "AC": [1200401], "AL": [2704302], "AM": [1302603], "AP": [1600303],"BA": [2927408], "CE": [2304400], "DF": [5300108], "ES": [3205309],"GO": [5208707], "MA": [2111300], "MG": [3106200], "MS": [5002704],
    "MT": [5103403], "PA": [1501402], "PB": [2507507], "PE": [2611606],"PI": [2211001], "PR": [4106902], "RJ": [3304557], "RN": [2408102],"RO": [1100205], "RR": [1400100], "RS": [4314902], "SC": [4205407],
    "SE": [2800308], "SP": [3550308], "TO": [1721000]
}

def baixar_dados_por_estado(diretorio_saida="data/raw", ano_inicio=2022, ano_fim=2025):
    os.makedirs(diretorio_saida, exist_ok=True)
    base_url = "https://info.dengue.mat.br/api/alertcity"
    dfs = []

    for estado, municipios in estados_ibge.items():
        for geocode in municipios:
            params = {
                "geocode": geocode,
                "disease": "dengue",
                "format": "csv",
                "ew_start": 1,
                "ew_end": 52,
                "ey_start": ano_inicio,
                "ey_end": ano_fim
            }
            url = f"{base_url}?{'&'.join([f'{k}={v}' for k, v in params.items()])}"
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text))
                    df["estado"] = estado
                    df["codigo_ibge"] = geocode
                    dfs.append(df)
                    print(f"✓ Dados coletados para o estado {estado}")
                else:
                    print(f"Erro ao coletar dados para o estado {estado}")
            except Exception as e:
                print(f"Erro com {estado}: {e}")

    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        df_final.to_csv(f"{diretorio_saida}/dengue_{ano_inicio}_{ano_fim}_por_estado.csv", index=False)
        print(f"\n✔ Dados salvos em {diretorio_saida}/dengue_{ano_inicio}_{ano_fim}_por_estado.csv")
    else:
        print("Nenhum dado foi coletado.")

if __name__ == "__main__":
    baixar_dados_por_estado(diretorio_saida="data/raw", ano_inicio=2022, ano_fim=2025)
